In [ ]:
!pip3 install apache_beam

**This is an example in which we will perform ETL on a csv file consisting of synthetic data(500,000 rows) of sales using Apache Beam using it's Python SDK.**

**Headers**
Region,Country,Item Type,Sales Channel,Order Priority,Order,Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit


**Case 1**
Get the total profit made by the *Item Type* s =[Clothes , Meat,Snacks,Fruits]




In [14]:
%%time
#importing apache beam module
import apache_beam as beam

#creating a beam pipeline object
p1 = beam.Pipeline()

#create list of required Item Types
selectedItemTypes = ["Clothes","Meat","Snacks","Fruits"]

#create a ParDo function to filter the records. You can also just use the beam.Filter()
class FilterItems(beam.DoFn):
  def process(self,element):
    if element[2] in selectedItemTypes:
      yield element

def SumFn(element):
    (key, row ) = element
    return (key,sum(row))

item_count = (
    p1
    | "Read csv file" >> beam.io.ReadFromText('SalesRecords.csv',skip_header_lines=1)
    | "Convert the rows in to arrays" >> beam.Map(lambda record:record.split(","))
    | "Filter on the givem Item Types" >> beam.ParDo(FilterItems())
    | "Create a tuple of (Item Type, Total Profit)" >> beam.Map(lambda record: (record[2],float(record[13])))
    | "collect up the profits" >> beam.GroupByKey()
    | "sum the profits" >> beam.Map(SumFn)
    | "print on terminal" >> beam.Map(print)
    | "writing op to text file" >> beam.io.WriteToText('test','end.txt',True,header=("Item Type","Total Profits"))
    )

#run the pipeline
p1.run()

('Fruits', 499972690.6800023)
('Clothes', 15332166989.279985)
('Meat', 11933838487.999947)
('Snacks', 11505215029.979986)
CPU times: user 6.62 s, sys: 54 ms, total: 6.67 s
Wall time: 6.86 s
